In [1]:
import praw
import json

In [2]:
def create_reddit_object(json_file='reddit_config.json',json_key='reddit_user_values'):
    with open(json_file) as f:
        data = json.load(f)
        
    user_values = data[json_key]
    
    reddit = praw.Reddit(client_id=user_values['client_id'],
                         client_secret=user_values['client_secret'],
                         user_agent=user_values['user_agent'],
                         username=user_values['username'],
                         password=user_values['password'])
    
    return reddit
        

In [3]:
def get_subreddit_data(subreddit, sort_type, time='all', limit=10):
#     top = subreddit.top("all",limit=100)
    attr = getattr(subreddit, sort_type)
    if sort_type=='top':
        data = attr(time, limit=limit)
    else:
        data = attr(limit=limit)
        
    top_df = pd.DataFrame()
    for submission in data:
        if submission.stickied:
            pass
        else:
            submission.comments.replace_more(limit=None)
            row = {}
            row['id']= submission.id
            row['title']= submission.title
            row['url']= submission.url
            row['author']= ['' if submission.author is None else submission.author.name][0]
            row['created']= pd.to_datetime(str(submission.created),unit='s')
            row['submission_text']= submission.selftext
            row['all_awardings']= submission.all_awardings
            row['category']= submission.category
            row['discussion_type']= submission.discussion_type
            row['distinguished']= submission.distinguished
            row['downs']= submission.downs
            row['gilded']= submission.gilded
            row['gildings']= submission.gildings
            row['likes']= submission.likes
            row['num_comments']= submission.num_comments
            row['num_crossposts']= submission.num_crossposts
            row['num_reports']= submission.num_reports
            row['score']= submission.score
            row['top_awarded_type']= submission.top_awarded_type
            row['total_awards_received']= submission.total_awards_received
            row['ups']= submission.ups
            row['upvote_ratio']= submission.upvote_ratio
            row['view_count']= submission.view_count

            top_levels = submission.comments[:]
            comments = [i.body for i in top_levels]
            row['comments']= comments

            comment_authors = [i.author for i in top_levels]
            comment_authors = ['' if comment_authors[i] is None else comment_authors[i].name for i in range(len(comment_authors))]
            row['cmnt_author']= comment_authors

            created = [pd.to_datetime(str(i.created),unit='s') for i in top_levels]
            row['cmnt_created'] = created
            row['cmnt_all_awardings'] = [i.all_awardings for i in top_levels] 
            row['cmnt_controversiality'] = [i.controversiality for i in top_levels]
            row['cmnt_downs'] = [i.downs for i in top_levels]
            row['cmnt_gilded'] = [i.gilded for i in top_levels]
            row['cmnt_gildings'] = [i.gildings for i in top_levels]
            row['cmnt_likes'] = [i.likes for i in top_levels]
            row['cmnt_num_reports'] = [i.num_reports for i in top_levels]
            row['cmnt_score'] = [i.score for i in top_levels]
            row['cmnt_top_awarded_type'] = [i.top_awarded_type for i in top_levels]
            row['cmnt_total_awards_received'] = [i.total_awards_received for i in top_levels]
            row['cmnt_ups'] = [i.ups for i in top_levels]


            rpl = [] 
            rpl_author = []
            rpl_created = []
            rpl_all_awardings = []
            rpl_controversiality = []
            rpl_downs = []
            rpl_gilded = []
            rpl_gildings = []
            rpl_likes = []
            rpl_num_reports = []
            rpl_score = []
            rpl_top_awarded_type = []
            rpl_total_awards_received = []
            rpl_ups = []
            for comment in top_levels:
                rpl.append([i.body for i in comment.replies])
                rpl_author.append([i.author for i in comment.replies])
                rpl_author[-1] = ['' if rpl_author[-1][i] is None else rpl_author[-1][i].name for i in range(len(rpl_author[-1]))]
                rpl_created.append([pd.to_datetime(str(i.created),unit='s') for i in comment.replies])
                rpl_all_awardings.append([i.all_awardings for i in comment.replies])
                rpl_controversiality.append([i.controversiality for i in comment.replies])
                rpl_downs.append([i.downs for i in comment.replies])
                rpl_gilded.append([i.gilded for i in comment.replies])
                rpl_gildings.append([i.gildings for i in comment.replies])
                rpl_likes.append([i.likes for i in comment.replies])
                rpl_num_reports.append([i.num_reports for i in comment.replies])
                rpl_score.append([i.score for i in comment.replies])
                rpl_top_awarded_type.append([i.top_awarded_type for i in comment.replies])
                rpl_total_awards_received.append([i.total_awards_received for i in comment.replies])
                rpl_ups.append([i.ups for i in comment.replies])

            row['replies'] = rpl
            row['rpl_author'] = rpl_author
            row['rpl_created'] = rpl_created
            row['rpl_all_awardings'] = rpl_all_awardings
            row['rpl_controversiality'] = rpl_controversiality
            row['rpl_downs'] = rpl_downs
            row['rpl_gilded'] = rpl_gilded
            row['rpl_gildings'] = rpl_gildings
            row['rpl_likes'] = rpl_likes
            row['rpl_num_reports'] = rpl_num_reports
            row['rpl_score'] = rpl_score
            row['rpl_top_awarded_type'] = rpl_top_awarded_type
            row['rpl_total_awards_received'] = rpl_total_awards_received
            row['rpl_ups'] = rpl_ups

            top_df = top_df.append(row, ignore_index=True)
            
    return top_df

In [4]:
fetch_new = False

In [5]:
import pandas as pd
from datetime import datetime
dt = datetime.today().strftime('%Y%m%d') 
filename = 'Discussion_top_all_100_20200604.json'

    
sub_title = 'Discussion'
sort_type = 'top'
time = 'all'
limit = 100

In [6]:
if not fetch_new:
    df = pd.read_json(filename)
else:
    reddit = create_reddit_object()
    subreddit = reddit.subreddit(sub_title)

    df = get_subreddit_data(subreddit, sort_type, time='all', limit=limit)
    df.to_json(sub_title + '_' + sort_type + '_' + time + '_' + str(limit) + '_' + dt + '.json')

In [7]:
# import nltk
# nltk.download('wordnet') # NEED TO INSTALL FOR LEMMATIZATION
# nltk.download('averaged_perceptron_tagger') # NEED TO INSTALL FOR POS ANALYSIS

In [8]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk import pos_tag, pos_tag_sents

In [9]:
from nltk.corpus import wordnet as wn

def is_noun(tag):
    return tag in ['NN', 'NNS', 'NNP', 'NNPS']

def is_verb(tag):
    return tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']

def is_adverb(tag):
    return tag in ['RB', 'RBR', 'RBS']

def is_adjective(tag):
    return tag in ['JJ', 'JJR', 'JJS']

def penn_to_wn(tag):
    if is_adjective(tag):
        return wn.ADJ
    elif is_adverb(tag):
        return wn.ADV
    elif is_verb(tag):
        return wn.VERB
    else:
        return wn.NOUN

In [10]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [45]:
sent_word_toks = []
lem = WordNetLemmatizer()
# ps = PorterStemmer()

for com in df['comments'].values[0]:
    sent_toks = sent_tokenize(com)
    for sent_tok in sent_toks:
        word_toks = [k for k in filter(lambda x: x not in stop_words, word_tokenize(sent_tok))]
        sent_word_toks.append(word_toks)

snt_wrd_pos_toks = pos_tag_sents(sent_word_toks)
snt_wrd_lem_toks = []
for i, tok_sent in enumerate(snt_wrd_pos_toks):
    tok_sent = [lem.lemmatize(word, penn_to_wn(pos)) for word,pos in tok_sent]
    snt_wrd_lem_toks.append(tok_sent)


In [48]:
snt_wrd_lem_toks

[['Yeah', ',', 'prank', 'culture', 'suck', '.'],
 ['Yeah', 'agree', '.'],
 ['I',
  'always',
  'see',
  'couple',
  'break',
  'prank',
  'cheat',
  'prank',
  'every',
  'often',
  'tell',
  'start',
  'affect',
  'trust',
  'real',
  'life',
  '.'],
 ['I',
  'get',
  'funny',
  'play',
  'joke',
  'eachother',
  'make',
  'hobby',
  'little',
  'immature',
  'start',
  'cause',
  'real',
  'issue',
  'relationship',
  '.'],
 ['Also',
  ',',
  'feel',
  'like',
  'couple',
  '’',
  'reaction',
  'obviously',
  'fake',
  'expect',
  'prank',
  'every',
  'week',
  'keep',
  'go',
  'along',
  'good',
  'content',
  '.'],
 ['yeah', '’', 'entertain', 'feel', 'bad', '😭😂', '(', 'RIP', 'BFvsGF', ')'],
 ['iTs', 'jUst', 'pRank', 'brO'],
 ['It',
  'seem',
  'like',
  "n't",
  'take',
  'seriously',
  'show',
  'others',
  '...',
  '.'],
 ['You', '’', '100', '%', 'right', ',', 'dude', '.'],
 ['Yah', 'also', '’', 'couple', 'Instagram', '...'],
 ['I',
  'forget',
  'name',
  'anyone',
  'know',
 

In [44]:
sent_word_toks

[['Yeah', ',', 'prank', 'culture', 'sucks', '.'],
 ['Yeah', 'agree', '.'],
 ['I',
  'always',
  'see',
  'couples',
  'break',
  'pranks',
  'cheating',
  'pranks',
  'every',
  'often',
  'tell',
  'starts',
  'affect',
  'trust',
  'real',
  'life',
  '.'],
 ['I',
  'get',
  'funny',
  'play',
  'jokes',
  'eachother',
  'making',
  'hobby',
  'little',
  'immature',
  'start',
  'cause',
  'real',
  'issues',
  'relationship',
  '.'],
 ['Also',
  ',',
  'feel',
  'like',
  'couples',
  '’',
  'reactions',
  'obviously',
  'fake',
  'expect',
  'pranked',
  'every',
  'week',
  'keep',
  'going',
  'along',
  'good',
  'content',
  '.'],
 ['yeah', '’', 'entertaining', 'feel', 'bad', '😭😂', '(', 'RIP', 'BFvsGF', ')'],
 ['iTs', 'jUst', 'pRank', 'brO'],
 ['It',
  'seems',
  'like',
  "n't",
  'take',
  'seriously',
  'shows',
  'others',
  '...',
  '.'],
 ['You', '’', '100', '%', 'right', ',', 'dude', '.'],
 ['Yah', 'also', '’', 'couple', 'Instagram', '...'],
 ['I',
  'forgot',
  'name',

In [37]:
sent_toks

[['Yeah, prank culture sucks.'],
 ['Yeah i agree.',
  'I always see couples doing break up pranks or cheating pranks on each other every so often and you can tell it starts to affect their trust in real life.',
  'I get it can be funny to play jokes on eachother here and there but making a hobby out of it can be a little immature and start to cause real issues in the relationship.',
  'Also, i feel like after a while the couples’ reactions are obviously fake as they expect to be pranked every week and just keep going along with it for good content.',
  'yeah they’re entertaining but i feel bad for them 😭😂(RIP BFvsGF)'],
 ['iTs jUst a pRank brO'],
 ["It just seems like they don't take each other seriously and shows others to do the same...."],
 ['You’re 100% right, my dude.'],
 ['Yah also there’s this on couple on Instagram ...',
  'I forgot the name but if anyone knows who I’m talking about please verify it if you want.',
  'I just see that these two are always yelling at each other sp